In [ ]:
left  = None
right = None
config1 = None
config2 = None
dem_gsd  = 24.0
img_gsd  = 6.0
output_path = None
maxdisp = None
downsample = None
pedr_list = None
# todo: add reference_dem and use to conditional pedr things

# Stereo Config file contents:

In [ ]:
if config2 == None:
    config2 = config1

In [ ]:
!cat {config1}

In [ ]:
!cat {config2}

# Setup Steps

In [ ]:
from IPython.display import Image
from pathlib import Path
from asap_stereo import asap
import math

In [ ]:
default_output_dir = '~/auto_asap/ctx/'
left, right = asap.CTX().get_ctx_order(left, right)
if output_path == None:
    output_path = default_output_dir + f'a_{left}_{right}'

In [ ]:
!mkdir -p {output_path}

In [ ]:
%cd {output_path}

# Download images (step 1)

In [ ]:
!asap ctx step-one {left} {right} 2>&1 | tee -i -a ./1_download.log ./full_log.log

# First Step of CTX processing lev1eo (Step 2)

In [ ]:
!asap ctx step-two  2>&1 | tee -i -a ./2_ctxedr2lev1eo.log ./full_log.log

In [ ]:
!asap ctx step-three

# Stereo Quality Report

In [ ]:
qual_report = asap.CommonSteps().get_stereo_quality_report(f'{left}_{right}/{left}.lev1eo.cub', f'{left}_{right}/{right}.lev1eo.cub')
print(qual_report)

# Downsample images if requested

In [ ]:
if downsample:
    true_img_gsd_left = asap.CommonSteps().get_image_gsd(f'{left}_{right}/{left}.lev1eo.cub')
    true_img_gsd_right = asap.CommonSteps().get_image_gsd(f'{left}_{right}/{right}.lev1eo.cub')
    # take conservative approach, pick worst image GSD
    res_gsd = max(true_img_gsd_left, true_img_gsd_right)
    # this is because rescale in ISIS does not update GSD in metadata
    asap.CommonSteps().rescale_and_overwrite(factor=downsample)
    img_gsd = math.ceil(res_gsd)*downsample
    dem_gsd = 4*img_gsd
    print('new img gsd', img_gsd)
    print('new dem gsd', dem_gsd)

# Calculate BA and low-res DEM (Step 3)

In [ ]:
!asap ctx step-four 2>&1 | tee -i -a ./2_bundle_adjust.log ./full_log.log

In [ ]:
!asap ctx step-five {config1}  2>&1 | tee -i -a ./3_lev1eo2dem.log ./full_log.log

In [ ]:
!asap ctx step-six {config1}  2>&1 | tee -i -a ./3_lev1eo2dem.log ./full_log.log

In [ ]:
!asap ctx step-seven --mpp 100 --just_dem True --dem_hole_fill_len 50 2>&1 | tee -i -a ./4_make_100m_dem.log ./full_log.log

In [ ]:
!asap ctx step-eight

## Good Pixel Map 

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_ba/{both}_ba-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

## Hillshade of low res DEM

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_ba/dem/{both}_ba_100_0-DEM-hillshade.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

# Mapproject ctx against 100m DEM

In [ ]:
!asap ctx step-nine --mpp {img_gsd} 2>&1 | tee -i -a ./5_mapproject_to_100m_dem.log ./full_log.log

# Calculate Better DEM using prior 

In [ ]:
!asap ctx step-ten {config2} 2>&1 | tee -i -a ./6_next_level_dem.log ./full_log.log

In [ ]:
!asap ctx step-eleven {config2} 2>&1 | tee -i -a ./6_next_level_dem.log ./full_log.log

In [ ]:
!asap ctx step-seven --mpp {dem_gsd} --folder results_map_ba

In [ ]:
!asap ctx step-eight --folder results_map_ba

# Get PEDR Shots for PC alignment (Step 5)

In [ ]:
!asap ctx step-twelve {pedr_list} 2>&1 | tee -i -a ./7_pedr_for_pc_align.log ./full_log.log


## Good Pixel Preview

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_map_ba/{both}_ba-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

## Hillshade of higher res DEM

In [ ]:
both = f'{left}_{right}'
img = './' + str(next(Path('./').glob(f'./{both}/results_map_ba/dem/{both}_ba_*-DEM-hillshade.tif')))
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

# Show pedr data

In [ ]:
!cat ./{left}_{right}/PEDR2TAB.PRM

In [ ]:
!cat ./{left}_{right}/{left}_{right}_pedr4align.csv | wc -l 

## start of PC align portion

In [ ]:
!asap ctx step_thirteen {maxdisp} 2>&1 | tee -i -a ./8_pc_align.log ./full_log.log

In [ ]:
!asap ctx step_fourteen --mpp {dem_gsd}  2>&1 | tee -i -a ./9_dems_orthos.log ./full_log.log

In [ ]:
!asap ctx step_fifteen 2>&1 | tee -i -a ./10_geoid_adjustment.log  ./full_log.log

In [ ]:
!asap ctx step_eight --folder results_map_ba --output_folder dem_align 2>&1 | tee -i -a ./11_hillshade.log ./full_log.log

In [ ]:
img = './' + str(next(Path('./').glob('./*/results_map_ba/dem_align/*_ba_align_*-DEM-hillshade.tif')))
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

In [ ]:
!asap ctx step_fourteen --mpp {img_gsd} --just_ortho True  2>&1 | tee -i -a ./12_img_full_ortho.log ./full_log.log